# **Bioinformatics with Jupyter Notebooks for WormBase:**
## **Analyses 1 - Alignment using BLAST**
Welcome to the fifth jupyter notebook in the WormBase tutorial series. Over this series of tutorials, we will write code in Python that allows us to retrieve and perform simple analyses with data available on the WormBase sites.

This tutorial will deal with performing BLAST alignment of your data against the WormBase Genome, ESTs and Protein data. 
Let's get started!

For this tutorial, we will use the wrappers for NCBI Blast+ application in the BioPython library. 

We will start with installing and importing the required libraries.

In [1]:
!pip install biopython

In [2]:
import Bio
import wget
import gzip 
import shutil
from Bio.Blast.Applications import NcbimakeblastdbCommandline, NcbiblastnCommandline, NcbiblastpCommandline
from Bio.Blast.Applications import NcbiblastxCommandline, NcbitblastnCommandline
from Bio.Blast import NCBIWWW, NCBIXML

### Creating a BLAST database

We need to first create our own BLAST database using the _C. elegans_ reference genome. (Or any reference genome based on your requirement). 
From the FTP site, we download the required reference genome and then generate the BLAST database with the wrapper for the NCBI BLAST+ `makeblastdb` command line.

Download the reference genome for the C. elegans species - BioProject PRJNA13758 and WormBase ID - WS280.
Check out tutorial 1 for more information on customizing your download.

In [3]:
#Generate the link for the reference genome
species = 'c_elegans'
bioproject = 'PRJNA13758'
wormbase_id = 'WS280'
descriptor = 'genomic'
extension = 'fa'
link = 'ftp://ftp.wormbase.org/pub/wormbase/releases/current-development-release/species/' + species + '/' + \
bioproject + '/' + species + '.' + bioproject + '.' + wormbase_id + '.' + descriptor + '.' + extension + '.gz'

#Download the reference genome
wget.download(link)
downloaded_file = species + '.' + bioproject + '.' + wormbase_id + '.' + descriptor + '.' + extension
#Unzip the reference genome to get the .fa file
with gzip.open(downloaded_file + '.gz', 'rb') as f_in:
    with open(downloaded_file, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

Create a commandline for the NCBI BLAST+ program makeblastdb, and then run the command.
Here, we create a nucleotide database using the reference genome downloaded in the previous cell.

In [4]:
command = NcbimakeblastdbCommandline(dbtype="nucl", 
                                     parse_seqids = 'TRUE', 
                                     input_file=downloaded_file, 
                                     out='worm_genome', 
                                     title='worm_genome')
command

NcbimakeblastdbCommandline(cmd='makeblastdb', out='worm_genome', dbtype='nucl', input_file='c_elegans.PRJNA13758.WS280.genomic.fa', title='worm_genome', parse_seqids=True)

In [5]:
command()

('\n\nBuilding a new DB, current time: 06/30/2021 03:42:16\nNew DB name:   /home/prajna/Research/Current/WormBase/worm_genome\nNew DB title:  worm_genome\nSequence type: Nucleotide\nKeep MBits: T\nMaximum file size: 1000000000B\nAdding sequences from FASTA; added 7 sequences in 1.04752 seconds.\n',
 '')

We have now created a new local database that we can use for our BLAST (blastn) alignments!!

### Running a BLAST query

We run blastn on our example.fa fasta file agaisnt the local BLAST database. The other parameters can be changed to your convinience.

Create a commandline for the NCBI BLAST+ program blastn and then run the command.

In [6]:
command = NcbiblastnCommandline(query="data/nucl_example.fa", #The sequence to search with
                                db="worm_genome", #The database to BLAST against
                                evalue=1e+0, #Expectation value cutoff
                                out="blastn.xml", #Output file for alignment
                                reward=1, #Reward for a nucleotide match 
                                penalty=-3, #Penalty for a nucleotide mismatch
                                outfmt='"5"', #Alignment view. -> 5 for XML
                                word_size=15, #Word size for wordfinder algorithm
                                gapopen=5, #Cost to open a gap
                                gapextend=2 #Cost to extend a gap
                               )

In [7]:
command()

('', '')

We now have generated an xml file with the results of the BLAST alignment and we will parse it to get the output in a readable and understandable format.

For this we use the NCBIXML module which can help us to easily parse the BLAST XML output.

In [8]:
result_handle = open("blastn.xml")

blast_records = NCBIXML.read(result_handle)

for i in range(len(blast_records.alignments)):
    for hsp in blast_records.alignments[i].hsps:
        print('Chromosome: ' + blast_records.descriptions[i].title.split(' ')[0])
        print(hsp)
        print('\n')

Chromosome: IV
Score 294 (583 bits), expectation 1.1e-165, alignment length 294
Query:     113 AGAAAGAGATGAACGTCTCAGACTTGATGTGTACCCGATCCTCGA...TGA 406
               |||||||||||||||||||||||||||||||||||||||||||||...|||
Sbjct: 2279181 AGAAAGAGATGAACGTCTCAGACTTGATGTGTACCCGATCCTCGA...TGA 2278888


Chromosome: IV
Score 93 (184 bits), expectation 9.8e-46, alignment length 97
Query:      16 CTTTTGAAGGTCAAAAAAGAGCCGCCGTCGAGTTCGGAAGAAGCC...GAA 112
               |||||| ||||||||||||||||||||||||||||||||||||||...|||
Sbjct: 2300226 CTTTTGCAGGTCAAAAAAGAGCCGCCGTCGAGTTCGGAAGAAGCC...GAA 2300130


Chromosome: IV
Score 50 (99 bits), expectation 4.5e-20, alignment length 50
Query:     401 AGGTGATCGGCAAAAAGTTCGTATATCGCTTTGTAACTACTGACG...CAC 450
               |||||||||||||||||||||||||||||||||||||||||||||...|||
Sbjct: 2278481 AGGTGATCGGCAAAAAGTTCGTATATCGCTTTGTAACTACTGACG...CAC 2278432


Chromosome: IV
Score 26 (52 bits), expectation 9.4e-06, alignment length 26
Query:       1 ATGAATCACATTGACCTTTTGAAGGT 26
 

### Example with blastn using ESTs 

We look at another example, performing blastn alignment with ESTs this time!

In [9]:
#Generate the link for the ests file
species = 'c_elegans'
bioproject = 'PRJNA13758'
wormbase_id = 'WS280'
descriptor = 'ests'
extension = 'fa'
link = 'ftp://ftp.wormbase.org/pub/wormbase/releases/current-development-release/species/' + species + \
'/' + bioproject + '/' + species + '.' + bioproject + '.' + wormbase_id + '.' + descriptor + '.' + extension + '.gz'

#Download the ests file
wget.download(link)
downloaded_file = species+'.'+bioproject+'.'+wormbase_id+'.'+descriptor+'.'+extension
#Unzip the ests file to get the .fa file
with gzip.open(downloaded_file + '.gz', 'rb') as f_in:
    with open(downloaded_file, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

In [10]:
command = NcbimakeblastdbCommandline(dbtype="nucl", 
                                     parse_seqids='TRUE', 
                                     input_file=downloaded_file, 
                                     out='worm_ests', 
                                     title='worm_ests')
command

NcbimakeblastdbCommandline(cmd='makeblastdb', out='worm_ests', dbtype='nucl', input_file='c_elegans.PRJNA13758.WS280.ests.fa', title='worm_ests', parse_seqids=True)

In [11]:
command()

('\n\nBuilding a new DB, current time: 06/30/2021 03:42:38\nNew DB name:   /home/prajna/Research/Current/WormBase/worm_ests\nNew DB title:  worm_ests\nSequence type: Nucleotide\nKeep MBits: T\nMaximum file size: 1000000000B\nAdding sequences from FASTA; added 402200 sequences in 9.00358 seconds.\n',
 '')

In [12]:
command = NcbiblastnCommandline(query="data/nucl_example.fa", #The sequence to search with
                                db="worm_ests", #The database to BLAST against
                                evalue=1e+0, #Expectation value cutoff
                                out="blastn_ests.xml", #Output file for alignment
                                reward=1, #Reward for a nucleotide match 
                                penalty=-3, #Penalty for a nucleotide mismatch
                                outfmt='"5"', #Alignment view. -> 5 for XML
                                word_size=15, #Word size for wordfinder algorithm
                                gapopen=5, #Cost to open a gap
                                gapextend=2 #Cost to extend a gap
                               )

In [13]:
command()

('', '')

In [14]:
result_handle = open("blastn_ests.xml")

blast_records = NCBIXML.read(result_handle)

for i in range(len(blast_records.alignments)):
    for hsp in blast_records.alignments[i].hsps:
        print('Chromosome: ' + blast_records.descriptions[i].title.split(' ')[0])
        print(hsp)
        print('\n')

Chromosome: gb|U38935|
Score 450 (892 bits), expectation 0.0e+00, alignment length 450
Query:       1 ATGAATCACATTGACCTTTTGAAGGTCAAAAAAGAGCCGCCGTCG...CAC 450
               |||||||||||||||||||||||||||||||||||||||||||||...|||
Sbjct:      49 ATGAATCACATTGACCTTTTGAAGGTCAAAAAAGAGCCGCCGTCG...CAC 498


Chromosome: yk1150h12.5
Score 450 (892 bits), expectation 0.0e+00, alignment length 450
Query:       1 ATGAATCACATTGACCTTTTGAAGGTCAAAAAAGAGCCGCCGTCG...CAC 450
               |||||||||||||||||||||||||||||||||||||||||||||...|||
Sbjct:      66 ATGAATCACATTGACCTTTTGAAGGTCAAAAAAGAGCCGCCGTCG...CAC 515


Chromosome: RST5_372761
Score 338 (670 bits), expectation 0.0e+00, alignment length 338
Query:     113 AGAAAGAGATGAACGTCTCAGACTTGATGTGTACCCGATCCTCGA...CAC 450
               |||||||||||||||||||||||||||||||||||||||||||||...|||
Sbjct:      23 AGAAAGAGATGAACGTCTCAGACTTGATGTGTACCCGATCCTCGA...CAC 360


Chromosome: emb|FM864439|
Score 338 (670 bits), expectation 0.0e+00, alignment length 338
Query:     113

### Example with blastp 

We look at another example, performing blastp alignment this time!

In [15]:
#Generate the link for the reference protein sequence
species = 'c_elegans'
bioproject = 'PRJNA13758'
wormbase_id = 'WS280'
descriptor = 'protein'
extension = 'fa'
link = 'ftp://ftp.wormbase.org/pub/wormbase/releases/current-development-release/species/' + species + '/' + 
bioproject + '/' + species + '.' + bioproject + '.' + wormbase_id + '.' + descriptor + '.' + extension + '.gz'

#Download the reference protein sequence
wget.download(link)
downloaded_file = species+'.'+bioproject+'.'+wormbase_id+'.'+descriptor+'.'+extension
#Unzip the reference protein sequence to get the .fa file
with gzip.open(downloaded_file + '.gz', 'rb') as f_in:
    with open(downloaded_file, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

In [16]:
command = NcbimakeblastdbCommandline(dbtype="prot", 
                                     parse_seqids='TRUE', 
                                     input_file=downloaded_file, 
                                     out='worm_protein', 
                                     title='worm_protein')
command

NcbimakeblastdbCommandline(cmd='makeblastdb', out='worm_protein', dbtype='prot', input_file='c_elegans.PRJNA13758.WS280.protein.fa', title='worm_protein', parse_seqids=True)

In [17]:
command()

('\n\nBuilding a new DB, current time: 06/30/2021 03:43:06\nNew DB name:   /home/prajna/Research/Current/WormBase/worm_protein\nNew DB title:  worm_protein\nSequence type: Protein\nKeep MBits: T\nMaximum file size: 1000000000B\nAdding sequences from FASTA; added 28389 sequences in 0.728467 seconds.\n',
 '')

In [18]:
command = NcbiblastpCommandline(query="data/prot_example.fa", #The sequence to search with
                                db="worm_protein", #The database to BLAST against
                                evalue=1e+0, #Expectation value cutoff
                                out="blastp.xml", #Output file for alignment
                                outfmt='"5"', #Alignment view. -> 5 for XML
                                gapopen=11, #Cost to open a gap
                                gapextend=1 #Cost to extend a gap
                               )

In [19]:
command()

('', '')

In [20]:
result_handle = open("blastp.xml")

blast_records = NCBIXML.read(result_handle)

for i in range(len(blast_records.alignments)):
    for hsp in blast_records.alignments[i].hsps:
        print('Chromosome: ' + blast_records.descriptions[i].title.split(' ')[0])
        print(hsp)
        print('\n')

Chromosome: C37F5.1a
Score 2311 (894 bits), expectation 0.0e+00, alignment length 441
Query:       1 MNHIDLLKVKKEPPSSSEEAEEEESPKHTIEGILDIRKKEMNVSD...PTL 441
               MNHIDLLKVKKEPPSSSEEAEEEESPKHTIEGILDIRKKEMNVSD...PTL
Sbjct:       1 MNHIDLLKVKKEPPSSSEEAEEEESPKHTIEGILDIRKKEMNVSD...PTL 441


Chromosome: C37F5.1b
Score 2096 (811 bits), expectation 0.0e+00, alignment length 401
Query:      41 MNVSDLMCTRSSTPPTSSSVDSIITLWQFLLELLQQDQNGDIIEW...PTL 441
               MNVSDLMCTRSSTPPTSSSVDSIITLWQFLLELLQQDQNGDIIEW...PTL
Sbjct:       1 MNVSDLMCTRSSTPPTSSSVDSIITLWQFLLELLQQDQNGDIIEW...PTL 401


Chromosome: Y73B3A.5
Score 1172 (456 bits), expectation 7.3e-162, alignment length 241
Query:     161 MNMKMCYVKDEKDIRHEIPSFMTSLQAPPPPPPPPQNPRGNTDFS...FSN 401
               MNMKMCYVKDEKDIRHEIPSFMTSLQAPPPPPPP QNPRGNTDFS...FSN
Sbjct:       1 MNMKMCYVKDEKDIRHEIPSFMTSLQAPPPPPPP-QNPRGNTDFS...FSN 237


Chromosome: C42D8.4a
Score 310 (124 bits), expectation 3.9e-33, alignment length 81
Query:      64 ITLWQFLLE

### Example with blastx

We look at another example, performing blastx alignment, nucleotide query against protein database, this time! 

In [21]:
#We will use the previously generated worm_protein database!!
command = NcbiblastxCommandline(query="data/nucl_example.fa", #The sequence to search with
                                db="worm_protein", #The database to BLAST against
                                evalue=1e+0, #Expectation value cutoff
                                out="blastx.xml", #Output file for alignment
                                outfmt='"5"', #Alignment view. -> 5 for XML
                                gapopen=11, #Cost to open a gap
                                gapextend=1 #Cost to extend a gap
                               )

In [22]:
command()

('', '')

In [23]:
result_handle = open("blastx.xml")

blast_records = NCBIXML.read(result_handle)

for i in range(len(blast_records.alignments)):
    for hsp in blast_records.alignments[i].hsps:
        print('Chromosome: ' + blast_records.descriptions[i].title.split(' ')[0])
        print(hsp)
        print('\n')

Chromosome: C37F5.1a
Score 654 (256 bits), expectation 2.9e-85, alignment length 150
Query:       1 MNHIDLLKVXXXXXXXXXXXXXXXXXXHTIEGILDIRKKEMNVSD...DAH 450
               MNHIDLLKVKKEPPSSSEEAEEEESPKHTIEGILDIRKKEMNVSD...DAH
Sbjct:       1 MNHIDLLKVKKEPPSSSEEAEEEESPKHTIEGILDIRKKEMNVSD...DAH 150


Chromosome: C37F5.1b
Score 558 (219 bits), expectation 2.4e-71, alignment length 110
Query:     121 MNVSDLMCTRXXXXXXXXXVDSIITLWQFLLELLQQDQNGDIIEW...DAH 450
               MNVSDLMCTRSSTPPTSSSVDSIITLWQFLLELLQQDQNGDIIEW...DAH
Sbjct:       1 MNVSDLMCTRSSTPPTSSSVDSIITLWQFLLELLQQDQNGDIIEW...DAH 110


Chromosome: C42D8.4a
Score 297 (119 bits), expectation 3.4e-34, alignment length 81
Query:     190 ITLWQFLLELLQQDQNGDIIEWTRGTDGEFRLIDAEAVARKWGQR...YRF 432
               I LWQFLLELL    N   I W  G++GEF+L+D + VARKWG+R...Y+F
Sbjct:      35 IQLWQFLLELLADAVNAHCIAW-EGSNGEFKLVDPDEVARKWGER...YKF 114


Chromosome: C42D8.4b
Score 298 (119 bits), expectation 4.9e-34, alignment length 81
Query:     190 ITLWQFLLELLQQD

### Example with tblastn

We look at another example, performing tblastn alignment, protein query against nucleotide database, this time! 

In [24]:
#We will use the previously generated worm_protein database!!
command = NcbitblastnCommandline(query="data/prot_example.fa", #The sequence to search with
                                db="worm_genome", #The database to BLAST against
                                evalue=1e+0, #Expectation value cutoff
                                out="tblastn.xml", #Output file for alignment
                                outfmt='"5"', #Alignment view. -> 5 for XML
                                gapopen=11, #Cost to open a gap
                                gapextend=1 #Cost to extend a gap
                               )

In [25]:
command()

('', '')

In [26]:
result_handle = open("tblastn.xml")

blast_records = NCBIXML.read(result_handle)

for i in range(len(blast_records.alignments)):
    for hsp in blast_records.alignments[i].hsps:
        print('Chromosome: ' + blast_records.descriptions[i].title.split(' ')[0])
        print(hsp)
        print('\n')

Chromosome: IV
Score 556 (218 bits), expectation 9.4e-62, alignment length 132
Query:     249 DESRQCRKRXXXXXXXXXXXXXXXXXXXXXKKGMKPNPLNLTATS...SQV 380
               DESRQCRKRSLSPSTTSSTTAPPPPPQPPTKKGMKPNPLNLTATS...SQV
Sbjct: 2277189 DESRQCRKRSLSPSTTSSTTAPPPPPQPPTKKGMKPNPLNLTATS...SQV 2277584


Chromosome: IV
Score 476 (187 bits), expectation 3.1e-51, alignment length 98
Query:      38 KKEMNVSDLMCTRXXXXXXXXXVDSIITLWQFLLELLQQDQNGDI...KKV 135
               +KEMNVSDLMCTRSSTPPTSSSVDSIITLWQFLLELLQQDQNGDI...KKV
Sbjct: 2278889 EKEMNVSDLMCTRSSTPPTSSSVDSIITLWQFLLELLQQDQNGDI...KKV 2279182


Chromosome: IV
Score 372 (147 bits), expectation 1.4e-37, alignment length 100
Query:     112 PHMNYDKLSRALRYYYEKNIIKKVIGKKFVYRFVTTDAHAPPTAD...LLG 211
               P  +  K  R   +   K  I +VIGKKFVYRFVTTDAHAPPTAD...LLG
Sbjct: 2278249 PKYDKRKRGRCTIFLLTKLNIFQVIGKKFVYRFVTTDAHAPPTAD...LLG 2278548


Chromosome: IV
Score 216 (87 bits), expectation 9.2e-18, alignment length 63
Query:     379 QVFQFPPVSAFQATNPLLNTFSNLISP

This is the end of the first tutorial for WormBase data analysis! This tutorial dealt with using BLAST alignment for any worm data.

In the next tutorial, we will use BLAT, which is a faster and more efficient version of BLAT for similar analyses!